In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 📈 Model Evaluation\n",
    "Comprehensive model evaluation and analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import os\n",
    "import sys\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.metrics import classification_report, confusion_matrix\n",
    "import tensorflow as tf\n",
    "from tensorflow import keras\n",
    "\n",
    "sys.path.append('..')\n",
    "from utils.data_loader import create_data_generators\n",
    "from utils.visualization import plot_confusion_matrix, plot_sample_predictions\n",
    "\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 8)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Load trained model\n",
    "MODEL_PATH = '../models/final/waste_classifier.h5'\n",
    "DATA_DIR = '../data/raw'\n",
    "\n",
    "model = keras.models.load_model(MODEL_PATH)\n",
    "print('✅ Model loaded successfully')\n",
    "model.summary()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Load Test Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Create test generator\n",
    "_, _, test_gen = create_data_generators(DATA_DIR)\n",
    "\n",
    "print(f'Test samples: {test_gen.samples}')\n",
    "print(f'Classes: {list(test_gen.class_indices.keys())}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Model Predictions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Get predictions\n",
    "y_pred_probs = model.predict(test_gen, verbose=1)\n",
    "y_pred = np.argmax(y_pred_probs, axis=1)\n",
    "y_true = test_gen.classes\n",
    "\n",
    "# Get class names\n",
    "class_names = list(test_gen.class_indices.keys())\n",
    "\n",
    "print(f'Predictions shape: {y_pred_probs.shape}')\n",
    "print(f'Number of predictions: {len(y_pred)}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Overall Metrics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Calculate overall accuracy\n",
    "accuracy = np.mean(y_pred == y_true)\n",
    "print(f'\\n🎯 Overall Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)')\n",
    "\n",
    "# Classification report\n",
    "report = classification_report(y_true, y_pred, target_names=class_names)\n",
    "print('\\n📊 Classification Report:')\n",
    "print(report)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Confusion Matrix"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Generate confusion matrix\n",
    "cm = confusion_matrix(y_true, y_pred)\n",
    "plot_confusion_matrix(cm, class_names)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Per-Class Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Calculate per-class metrics\n",
    "per_class_metrics = []\n",
    "\n",
    "for i, class_name in enumerate(class_names):\n",
    "    class_mask = y_true == i\n",
    "    class_acc = np.mean(y_pred[class_mask] == y_true[class_mask])\n",
    "    class_samples = np.sum(class_mask)\n",
    "    \n",
    "    per_class_metrics.append({\n",
    "        'Class': class_name,\n",
    "        'Accuracy': class_acc,\n",
    "        'Samples': class_samples,\n",
    "        'Correct': np.sum(y_pred[class_mask] == y_true[class_mask]),\n",
    "        'Incorrect': np.sum(y_pred[class_mask] != y_true[class_mask])\n",
    "    })\n",
    "\n",
    "df_per_class = pd.DataFrame(per_class_metrics)\n",
    "df_per_class = df_per_class.sort_values('Accuracy', ascending=False)\n",
    "\n",
    "print('\\n📊 Per-Class Performance:')\n",
    "print(df_per_class.to_string(index=False))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Plot per-class accuracy\n",
    "plt.figure(figsize=(12, 6))\n",
    "bars = plt.bar(df_per_class['Class'], df_per_class['Accuracy'], color='steelblue', edgecolor='black')\n",
    "\n",
    "# Color bars based on accuracy\n",
    "for i, bar in enumerate(bars):\n",
    "    if df_per_class.iloc[i]['Accuracy'] >= 0.9:\n",
    "        bar.set_color('green')\n",
    "    elif df_per_class.iloc[i]['Accuracy'] >= 0.7:\n",
    "        bar.set_color('orange')\n",
    "    else:\n",
    "        bar.set_color('red')\n",
    "\n",
    "plt.axhline(y=accuracy, color='blue', linestyle='--', label=f'Overall Accuracy: {accuracy:.3f}')\n",
    "plt.title('Per-Class Accuracy', fontsize=16, fontweight='bold')\n",
    "plt.xlabel('Class', fontsize=12)\n",
    "plt.ylabel('Accuracy', fontsize=12)\n",
    "plt.ylim([0, 1])\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "plt.legend()\n",
    "plt.grid(axis='y', alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Confidence Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Analyze prediction confidence\n",
    "max_probs = np.max(y_pred_probs, axis=1)\n",
    "correct_mask = (y_pred == y_true)\n",
    "\n",
    "correct_confidence = max_probs[correct_mask]\n",
    "incorrect_confidence = max_probs[~correct_mask]\n",
    "\n",
    "print(f'\\n📈 Confidence Analysis:')\n",
    "print(f'Correct predictions - Mean confidence: {correct_confidence.mean():.4f}')\n",
    "print(f'Incorrect predictions - Mean confidence: {incorrect_confidence.mean():.4f}')\n",
    "\n",
    "# Plot confidence distribution\n",
    "fig, ax = plt.subplots(1, 1, figsize=(12, 6))\n",
    "\n",
    "ax.hist(correct_confidence, bins=50, alpha=0.7, label='Correct', color='green', edgecolor='black')\n",
    "ax.hist(incorrect_confidence, bins=50, alpha=0.7, label='Incorrect', color='red', edgecolor='black')\n",
    "ax.set_xlabel('Confidence', fontsize=12)\n",
    "ax.set_ylabel('Frequency', fontsize=12)\n",
    "ax.set_title('Prediction Confidence Distribution', fontsize=14, fontweight='bold')\n",
    "ax.legend()\n",
    "ax.grid(alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Error Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Find most common misclassifications\n",
    "errors = []\n",
    "\n",
    "for i in range(len(class_names)):\n",
    "    for j in range(len(class_names)):\n",
    "        if i != j:\n",
    "            count = np.sum((y_true == i) & (y_pred == j))\n",
    "            if count > 0:\n",
    "                errors.append({\n",
    "                    'True Class': class_names[i],\n",
    "                    'Predicted Class': class_names[j],\n",
    "                    'Count': count\n",
    "                })\n",
    "\n",
    "df_errors = pd.DataFrame(errors)\n",
    "df_errors = df_errors.sort_values('Count', ascending=False)\n",
    "\n",
    "print('\\n❌ Top 10 Misclassifications:')\n",
    "print(df_errors.head(10).to_string(index=False))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Sample Predictions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Display sample predictions\n",
    "plot_sample_predictions(model, test_gen, num_samples=16)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. Model Summary Report"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Generate summary report\n",
    "report_data = {\n",
    "    'Metric': [\n",
    "        'Test Accuracy',\n",
    "        'Test Loss',\n",
    "        'Total Samples',\n",
    "        'Correct Predictions',\n",
    "        'Incorrect Predictions',\n",
    "        'Mean Confidence (Correct)',\n",
    "        'Mean Confidence (Incorrect)',\n",
    "        'Number of Classes'\n",
    "    ],\n",
    "    'Value': [\n",
    "        f'{accuracy:.4f}',\n",
    "        f'{model.evaluate(test_gen, verbose=0)[0]:.4f}',\n",
    "        len(y_true),\n",
    "        np.sum(correct_mask),\n",
    "        np.sum(~correct_mask),\n",
    "        f'{correct_confidence.mean():.4f}',\n",
    "        f'{incorrect_confidence.mean():.4f}' if len(incorrect_confidence) > 0 else 'N/A',\n",
    "        len(class_names)\n",
    "    ]\n",
    "}\n",
    "\n",
    "df_report = pd.DataFrame(report_data)\n",
    "\n",
    "print('\\n' + '='*50)\n",
    "print('📊 MODEL EVALUATION SUMMARY')\n",
    "print('='*50)\n",
    "print(df_report.to_string(index=False))\n",
    "print('='*50)\n",
    "\n",
    "# Save report\n",
    "df_report.to_csv('../models/evaluation_summary.csv', index=False)\n",
    "print('\\n✅ Evaluation summary saved to: models/evaluation_summary.csv')"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}